# Debugging Subject 1 Insole Data Issues in the CPAC_S01_S02_S03_05_31_20 dataset

## Libraries

In [1]:
import sklearn
assert sklearn.__version__ >= "0.21", "Use the conda_python3_latest kernel!"

# Standard library
import os
import re
import warnings

# Third party
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn import (dummy, linear_model, ensemble, 
                     metrics, preprocessing, pipeline, inspection, 
                     model_selection)

from IPython.display import display, Markdown

# Local
import utils

# matplotlib hackery
%matplotlib inline

## Load Dataset

In [2]:
df_orig = utils.load_dataset("s3://cpac/ORIG/CPAC_S01_S02_S03_05_31_20/CPAC10S_S01_S02_S03_06_01_20.csv")
df_orig.describe()

,M_Trial_Num,M_Mass,M_Mass_to_L5S1,M_sub_task_indices,M_sub_task_num,M_include_overall,M_Index,M_Sub,M_sub_task_num_overall,M_Index_overall,...,RWEO_03_04_00_00_INSOLE_LY_AP_threshF50_mm,RWEO_01_00_00_00_INSOLE_RFORCE_threshF50_N,RWEO_01_02_00_00_INSOLE_RX_ML_threshF50_mm,RWEO_01_02_00_00_INSOLE_RY_AP_threshF50_mm,RWEF_03_00_00_00_INSOLE_LFORCE_threshF50_BW,RWEF_03_04_00_00_INSOLE_LX_ML_threshF50_BH,RWEF_03_04_00_00_INSOLE_LY_AP_threshF50_BH,RWEF_01_00_00_00_INSOLE_RFORCE_threshF50_BW,RWEF_01_02_00_00_INSOLE_RX_ML_threshF50_BH,RWEF_01_02_00_00_INSOLE_RY_AP_threshF50_BH
count,596920.000000,596920.000000,450392.000000,596920.000000,596920.000000,596920.000000,596920.000000,596920.000000,596920.000000,596920.000000,...,509406.000000,594981.000000,506937.000000,506937.000000,594981.000000,509406.000000,509406.000000,594981.000000,506937.000000,506937.000000
mean,68.172854,10.576766,0.310355,295.212494,5.496341,0.782267,2850.599620,1.897787,235.306545,76305.094906,...,121.034549,357.312319,47.795679,135.603684,0.473457,0.028690,0.067842,0.487438,0.026754,0.075886
std,23.458992,5.960748,0.164044,234.100540,4.754427,0.412705,2708.406675,0.817537,121.308144,72015.333181,...,55.562888,283.629719,8.805979,53.366872,0.336114,0.005458,0.031498,0.354233,0.004836,0.030041
min,1.000000,0.000000,0.083662,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,-0.195016,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,49.000000,5.000000,0.154197,121.000000,1.000000,1.000000,603.000000,1.000000,132.000000,0.000000,...,75.000000,125.815000,44.110000,93.770000,0.173023,0.026911,0.041779,0.187174,0.024577,0.052399
50%,78.000000,10.000000,0.274311,250.000000,4.000000,1.000000,1954.000000,2.000000,250.000000,63184.500000,...,112.680000,313.860000,49.980000,133.750000,0.482852,0.029677,0.062232,0.481379,0.028043,0.073984
75%,85.000000,15.000000,0.457227,407.000000,8.000000,1.000000,4445.000000,3.000000,335.000000,137799.250000,...,168.120000,559.535000,53.710000,183.140000,0.726662,0.032168,0.093348,0.726368,0.029920,0.101509
max,96.000000,23.000000,0.701759,1866.000000,21.000000,1.000000,14119.000000,3.000000,452.000000,233105.000000,...,242.320000,1558.065000,71.260000,247.560000,1.949560,0.041882,0.140069,2.041016,0.041191,0.143098


## Associate column names

In [3]:
def _get_columns_with_prefix(df, prefix):
    columns = []
    for column in df.columns:
        if column.startswith(prefix):
            columns.append(column)
    return columns
    
def get_target_names(df):
    return _get_columns_with_prefix(df, "T_")

def get_meta_names(df):
    return _get_columns_with_prefix(df, "M_")    

## Clean-up dataset

- Remove samples based on `M_include_overall`

In [4]:
df = df_orig[df_orig["M_include_overall"] > 0]
print(f"Number of samples: {df.shape[0]} (before clean-up: {df_orig.shape[0]})")
print(f"Number of trials: {len(df['M_Trial_Name'].unique())} (before clean-up: {len(df_orig['M_Trial_Name'].unique())})")
print(f"Number of subjects: {len(df['M_Sub'].unique())}")

Number of samples: 466951 (before clean-up: 596920)
Number of trials: 140 (before clean-up: 141)
Number of subjects: 3


## Predictor configurations (recipes)

In [5]:
def predictor_short_name(predictor):
    return predictor[17:]

def predictor_sensor_number(predictor):
    return int(predictor[5:7])

def filter_predictors(all_predictors, patterns):
    if isinstance(patterns, str):
        patterns = (patterns,)
        
    predictors = []
    for predictor in all_predictors:
        for pattern in patterns:
            if pattern in predictor:
                predictors.append(predictor)
                break
    return predictors


feature_sets = {
    "Recipe 1: Simulated Wearable - All Regular": 
        filter_predictors(df.columns, "SWRF"),
    
    "Recipe 2: Simulated Wearable - All Regular & Exploratory": 
        filter_predictors(df.columns, ("SWRF", "SWEF")),
    
    "Recipe 3: Real Wearable - All Regular": 
        filter_predictors(df.columns, "RWRF"),
    
    "Recipe 4: Real Wearable - All Regular & Exploratory": 
        filter_predictors(df.columns, ("RWRF", "RWEF")),
    
    "Recipe 5: Real Wearable - Insole Only":
        filter_predictors(
            filter_predictors(df.columns, ("RWRF", "RWEF")),
            ("03_00_00_00", "01_00_00_00", "03_04_00_00", "01_02_00_00", "01_03_00_00")
        ),
    
    "Recipe 6: Real Wearable - Trunk Orientatoin Only":
        filter_predictors(
            filter_predictors(df.columns, ("RWRF", "RWEF")),
            ("12_00_00_00",)
        ),
    
    "Recipe 7: Real Wearable - Insole & Trunk Orientation":
        filter_predictors(
            filter_predictors(df.columns, ("RWRF", "RWEF")),
            ("03_00_00_00", "01_00_00_00", "03_04_00_00", "01_02_00_00", "01_03_00_00", 
             "12_00_00_00", "01_03_12_00")
        ),
    
    "Recipe 8: Real Wearable - Insole & Hip Angles & Trunk Orientation":
        filter_predictors(
            filter_predictors(df.columns, ("RWRF", "RWEF")),
            ("03_00_00_00", "01_00_00_00", "03_04_00_00", "01_02_00_00", "01_03_00_00", 
             "12_00_00_00", "01_03_12_00",
             "05_06_00_00", "05_09_00_00", "06_00_00_00", "09_00_00_00", "05_12_00_00", "05_06_01_03", "05_09_01_03")
        ),
    
}

for feature_set_name, predictors in feature_sets.items():
    sensors = set(map(predictor_sensor_number, predictors))
    print(f"{feature_set_name}\n\tPredictors: {len(predictors)}, Sensors: {len(sensors)}\n")

Recipe 1: Simulated Wearable - All Regular
	Predictors: 75, Sensors: 10

Recipe 2: Simulated Wearable - All Regular & Exploratory
	Predictors: 104, Sensors: 10

Recipe 3: Real Wearable - All Regular
	Predictors: 166, Sensors: 10

Recipe 4: Real Wearable - All Regular & Exploratory
	Predictors: 201, Sensors: 10

Recipe 5: Real Wearable - Insole Only
	Predictors: 17, Sensors: 2

Recipe 6: Real Wearable - Trunk Orientatoin Only
	Predictors: 56, Sensors: 1

Recipe 7: Real Wearable - Insole & Trunk Orientation
	Predictors: 77, Sensors: 3

Recipe 8: Real Wearable - Insole & Hip Angles & Trunk Orientation
	Predictors: 133, Sensors: 6



## Train/Test configurations

In [6]:
train_test_selectors = {
    "Subject 1 & 2 -> Subject 3": (df["M_Sub"] != 3, df["M_Sub"] == 3),
    "Subject 1 & 3 -> Subject 2": (df["M_Sub"] != 2, df["M_Sub"] == 2),
    "Subject 2 & 3 -> Subject 1": (df["M_Sub"] != 1, df["M_Sub"] == 1),
    "Subject 1 -> Subject 2": (df["M_Sub"] == 1, df["M_Sub"] == 2),
    "Subject 2 -> Subject 1": (df["M_Sub"] == 2, df["M_Sub"] == 1),
    "Subject 1 -> Subject 3": (df["M_Sub"] == 1, df["M_Sub"] == 3),
    "Subject 3 -> Subject 1": (df["M_Sub"] == 3, df["M_Sub"] == 1),
    "Subject 2 -> Subject 3": (df["M_Sub"] == 2, df["M_Sub"] == 3),
    "Subject 3 -> Subject 2": (df["M_Sub"] == 3, df["M_Sub"] == 2)
}

train_test_sets = {}
for test_selector_name, (train_selector, test_selector) in train_test_selectors.items():
    #df_train, df_test = df[train_selector].dropna(), df[test_selector].dropna()
    df_train, df_test = df[train_selector], df[test_selector]
    train_test_sets[test_selector_name] = (df_train, df_test)
    print(f"{test_selector_name}: {len(df_train)}/{len(df_test)} "
          f"({len(df_train) / len(df):.1%}/{len(df_test) / len(df):.1%})")

Subject 1 & 2 -> Subject 3: 317564/149387 (68.0%/32.0%)
Subject 1 & 3 -> Subject 2: 308407/158544 (66.0%/34.0%)
Subject 2 & 3 -> Subject 1: 307931/159020 (65.9%/34.1%)
Subject 1 -> Subject 2: 159020/158544 (34.1%/34.0%)
Subject 2 -> Subject 1: 158544/159020 (34.0%/34.1%)
Subject 1 -> Subject 3: 159020/149387 (34.1%/32.0%)
Subject 3 -> Subject 1: 149387/159020 (32.0%/34.1%)
Subject 2 -> Subject 3: 158544/149387 (34.0%/32.0%)
Subject 3 -> Subject 2: 149387/158544 (32.0%/34.0%)


## Train and evaluate boosted tree models

In [7]:
def train_model(X_train, y_train):
    model = pipeline.Pipeline([
        ('scaler', preprocessing.StandardScaler()),
        ('gboost', ensemble.HistGradientBoostingRegressor())
    ]).fit(X_train, y_train)
    return model
    
def evaluate_model(desc, model, X_test, y_test, show_plot=False, show_importance=False):
    y_test_pred = model.predict(X_test)
    
    # Quantitative results
    rmse = np.sqrt(metrics.mean_squared_error(y_test_pred, y_test))
    r2 = metrics.r2_score(y_test_pred, y_test)
    
    display(
        Markdown(
            "---\n"
            f"**Target**: {desc['target']}  \n"
            f"**Features**: {desc['feature_set']}  \n"
            f"**Train/Test**: {desc['train_test_set']}  \n"
            f"**R2={r2:.3f}**"
        )
    )
    
    if show_plot:
        ax = sns.jointplot(y_test, y_test_pred, alpha=0.1, s=1.0, color=desc["color"])
        ax.set_axis_labels("Actual Pelvis Moment", 'Predicted Pelvis Moment', fontsize=14)
        ax.ax_joint.grid()
        ax.ax_marg_x.set_title("Predictions")

        summary = '\n'.join((
            r"$\mathrm{RMSE}=%.2f$" % (rmse,),
            r"$\mathrm{R}^2=%.3f$" % (r2,)
        ))

        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        ax.ax_joint.text(0.05, 0.95, summary, transform=ax.ax_joint.transAxes, fontsize=14,
            verticalalignment='top', bbox=props)
        plt.show()
        
    if show_importance:
        feature_importances = inspection.permutation_importance(model, 
                                           X_test, 
                                           y_test, 
                                           n_repeats=5, n_jobs=-1)
        top_idxs = feature_importances.importances_mean.argsort()[::-1][:6]
        fig, ax = plt.subplots()
#         ax.boxplot(feature_importances.importances[top_idxs].T,
#                vert=False, labels=[predictor_short_name(X_test.columns[idx]) for idx in top_idxs])
        ax.barh(
            [predictor_short_name(X_test.columns[idx]) for idx in top_idxs], 
            feature_importances.importances_mean[top_idxs],
            color=desc["color"]
        )
        ax.set_title("Feature Importances")
        plt.show()
    
    return r2, pd.Series(y_test_pred, index=y_test.index)

def run_experiments(target_name, train_test_sets, color="steelblue"):
    warnings.filterwarnings('ignore')
    r2_scores = {}
    predictions = {}
    for train_test_set_name, (df_train, df_test) in train_test_sets.items():
        for feature_set_name, feature_names in feature_sets.items():
            desc = {"target": target_name, 
                    "feature_set": feature_set_name, 
                    "train_test_set": train_test_set_name,
                    "color": color
                   }
            model = train_model(df_train[feature_names], df_train[target_name])
            r2, prediction = evaluate_model(
                desc, 
                model, 
                df_test[feature_names], 
                df_test[target_name], 
                show_plot=False, 
                show_importance=False)
            
            r2_scores.setdefault(train_test_set_name, {})[feature_set_name] = r2
            predictions.setdefault(train_test_set_name, {})[feature_set_name] = prediction
    warnings.filterwarnings('default')
    return r2_scores, predictions


## Estimating TF_Pelvis_Moment_X_BWBH


In [8]:
r2_scores_X, predictions_X = run_experiments("TF_Pelvis_Moment_X_BWBH", train_test_sets, color="steelblue")

---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 1: Simulated Wearable - All Regular  
**Train/Test**: Subject 1 & 2 -> Subject 3  
**R2=0.903**

---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 2: Simulated Wearable - All Regular & Exploratory  
**Train/Test**: Subject 1 & 2 -> Subject 3  
**R2=0.904**

---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 3: Real Wearable - All Regular  
**Train/Test**: Subject 1 & 2 -> Subject 3  
**R2=0.748**

---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 4: Real Wearable - All Regular & Exploratory  
**Train/Test**: Subject 1 & 2 -> Subject 3  
**R2=0.761**

---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 5: Real Wearable - Insole Only  
**Train/Test**: Subject 1 & 2 -> Subject 3  
**R2=-0.953**

...

---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 4: Real Wearable - All Regular & Exploratory  
**Train/Test**: Subject 3 -> Subject 2  
**R2=0.573**

---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 5: Real Wearable - Insole Only  
**Train/Test**: Subject 3 -> Subject 2  
**R2=-3.971**

---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 6: Real Wearable - Trunk Orientatoin Only  
**Train/Test**: Subject 3 -> Subject 2  
**R2=0.510**

---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 7: Real Wearable - Insole & Trunk Orientation  
**Train/Test**: Subject 3 -> Subject 2  
**R2=0.590**

---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 8: Real Wearable - Insole & Hip Angles & Trunk Orientation  
**Train/Test**: Subject 3 -> Subject 2  
**R2=0.633**

## Estimating TF_Pelvis_Moment_Y_BWBH


In [ ]:
r2_scores_Y, predictions_Y = run_experiments("TF_Pelvis_Moment_Y_BWBH", train_test_sets, color="darkorange")

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 1: Simulated Wearable - All Regular  
**Train/Test**: Subject 1 & 2 -> Subject 3  
**R2=0.900**

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 2: Simulated Wearable - All Regular & Exploratory  
**Train/Test**: Subject 1 & 2 -> Subject 3  
**R2=0.906**

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 3: Real Wearable - All Regular  
**Train/Test**: Subject 1 & 2 -> Subject 3  
**R2=0.722**

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 4: Real Wearable - All Regular & Exploratory  
**Train/Test**: Subject 1 & 2 -> Subject 3  
**R2=0.714**

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 5: Real Wearable - Insole Only  
**Train/Test**: Subject 1 & 2 -> Subject 3  
**R2=0.625**

...

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 4: Real Wearable - All Regular & Exploratory  
**Train/Test**: Subject 3 -> Subject 2  
**R2=0.300**

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 5: Real Wearable - Insole Only  
**Train/Test**: Subject 3 -> Subject 2  
**R2=-1.052**

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 6: Real Wearable - Trunk Orientatoin Only  
**Train/Test**: Subject 3 -> Subject 2  
**R2=0.413**

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 7: Real Wearable - Insole & Trunk Orientation  
**Train/Test**: Subject 3 -> Subject 2  
**R2=0.154**

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 8: Real Wearable - Insole & Hip Angles & Trunk Orientation  
**Train/Test**: Subject 3 -> Subject 2  
**R2=0.337**

## Estimating TF_Pelvis_Moment_X_BWBH on a limited target range

In [ ]:
train_test_sets_limited = {}
for test_selector_name, (train_selector, test_selector) in train_test_selectors.items():
    test_selector_name += "_limited"
    #df_train, df_test = df[train_selector].dropna(), df[test_selector].dropna()
    df_train, df_test = df[train_selector], df[test_selector]
    df_train = df_train[df_train["TF_Pelvis_Moment_X_BWBH"].between(-0.2, -0.02)]
    df_test = df_test[df_test["TF_Pelvis_Moment_X_BWBH"].between(-0.2, -0.02)]
    train_test_sets_limited[test_selector_name] = (df_train, df_test)
    print(f"{test_selector_name}: {len(df_train)}/{len(df_test)} "
          f"({len(df_train) / len(df):.1%}/{len(df_test) / len(df):.1%})")
    
r2_scores_X_limited, predictions_X_limited = run_experiments("TF_Pelvis_Moment_X_BWBH", train_test_sets_limited, color="green")

Subject 1 & 2 -> Subject 3_limited: 219052/100352 (46.9%/21.5%)
Subject 1 & 3 -> Subject 2_limited: 213600/105804 (45.7%/22.7%)
Subject 2 & 3 -> Subject 1_limited: 206156/113248 (44.1%/24.3%)
Subject 1 -> Subject 2_limited: 113248/105804 (24.3%/22.7%)
Subject 2 -> Subject 1_limited: 105804/113248 (22.7%/24.3%)
Subject 1 -> Subject 3_limited: 113248/100352 (24.3%/21.5%)
Subject 3 -> Subject 1_limited: 100352/113248 (21.5%/24.3%)
Subject 2 -> Subject 3_limited: 105804/100352 (22.7%/21.5%)
Subject 3 -> Subject 2_limited: 100352/105804 (21.5%/22.7%)


---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 1: Simulated Wearable - All Regular  
**Train/Test**: Subject 1 & 2 -> Subject 3_limited  
**R2=0.821**

---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 2: Simulated Wearable - All Regular & Exploratory  
**Train/Test**: Subject 1 & 2 -> Subject 3_limited  
**R2=0.825**

---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 3: Real Wearable - All Regular  
**Train/Test**: Subject 1 & 2 -> Subject 3_limited  
**R2=0.532**

---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 4: Real Wearable - All Regular & Exploratory  
**Train/Test**: Subject 1 & 2 -> Subject 3_limited  
**R2=0.520**

...

---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 4: Real Wearable - All Regular & Exploratory  
**Train/Test**: Subject 3 -> Subject 2_limited  
**R2=0.590**

---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 5: Real Wearable - Insole Only  
**Train/Test**: Subject 3 -> Subject 2_limited  
**R2=-3.719**

---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 6: Real Wearable - Trunk Orientatoin Only  
**Train/Test**: Subject 3 -> Subject 2_limited  
**R2=0.005**

---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 7: Real Wearable - Insole & Trunk Orientation  
**Train/Test**: Subject 3 -> Subject 2_limited  
**R2=0.503**

---
**Target**: TF_Pelvis_Moment_X_BWBH  
**Features**: Recipe 8: Real Wearable - Insole & Hip Angles & Trunk Orientation  
**Train/Test**: Subject 3 -> Subject 2_limited  
**R2=0.537**

## Saving results

In [ ]:
with pd.ExcelWriter('CPAC_S01_S02_S03_05_31_20 - Subject 1 Insole Issues.xlsx') as writer:
    pd.DataFrame(r2_scores_X).to_excel(writer, sheet_name='TF_Pelvis_Moment_X_BWBH')
    pd.DataFrame(r2_scores_X_limited).to_excel(writer, sheet_name='TF_Pelvis_Moment_X_BWBH_limited')
    pd.DataFrame(r2_scores_Y).to_excel(writer, sheet_name='TF_Pelvis_Moment_Y_BWBH')